In [1]:
import numpy as np
import pandas as pd

年数据

In [2]:
#读取年数据
data_year = pd.read_csv('Data/年数据.csv',encoding = 'GBK',engine='python')
data_year.head()

,股票编号,年份（年末）,固定资产合计,无息流动负债,无息非流动负债,带息流动负债,带息债务,净债务,有形净资产,营运资本,...,现金及现金等价物净增加额,加:期初现金及现金等价物余额,现金及现金等价物净增加额的特殊项目,现金及现金等价物净增加额的调整金额,期末现金及现金等价物余额,高转送预案公告日,高转送股权登记日,高转送除权日,每股送转,是否高转送
0,1,1,86912289.26,1.422495e+09,160019158.3,819855100.3,827188433.6,357874692.1,8.929308e+08,5.904970e+08,...,-7.615285e+07,545466594.5,NaN,NaN,469313741.6,3月30日,NaN,NaN,NaN,0
1,1,2,78878168.21,1.903724e+09,148736391.3,374909888.3,394226555.0,-403497756.4,1.190906e+09,9.114353e+08,...,3.284106e+08,469313741.6,NaN,NaN,797724311.4,3月30日,NaN,NaN,NaN,0
2,1,3,75301015.72,1.447218e+09,141831622.4,364316666.6,480560018.6,-496611795.6,1.501162e+09,1.333070e+09,...,1.794475e+08,797724311.4,NaN,NaN,977171814.2,4月1日,6月19日,6月20日,0.25,0
3,1,4,64069233.96,1.388840e+09,136730134.5,105000000.0,282613352.0,-526350024.7,1.755344e+09,1.697810e+09,...,-1.682084e+08,977171814.2,NaN,NaN,808963376.7,4月1日,6月8日,6月9日,0.22,0
4,1,5,85929516.37,1.870206e+09,134704875.2,129243352.0,274083358.8,-671656616.9,1.764907e+09,1.665822e+09,...,1.243744e+08,808963376.7,NaN,NaN,933337815.8,3月31日,5月23日,5月24日,0.08,0


标签和时序因子的获取

In [3]:
# 时序因子，本年送转比例
tran_data = data_year.iloc[:,:2]
tran_data['本年送转比例'] = data_year['每股送转']
# 填补缺失值
tran_data.fillna(0,inplace = True)

tran_data['是否送转'] = 1 * (data_year['每股送转'] > 0) 
tran_data['是否高送转'] = 1 * (data_year['每股送转'] >= 0.5) 

In [7]:
def get_near2_tran(code,year):
    """
        input:
            code:股票编号
            year:年份
        output:
            该股票该年份近两年的送转比例平均值
    """
    if year == 1:
        return None
    info = tran_data[tran_data['股票编号'] == code]
    rate = info[info['年份（年末）'] == year]['本年送转比例'].values[0] + info[info['年份（年末）'] == year-1]['本年送转比例'].values[0]
    return rate/2

def get_next_tran(code,year):
    """
    input:
        code:股票编号
        year:年份
    output:
        该股票在对应年份的次年是否送转
        
    """
    info = tran_data[tran_data['股票编号'] == code]
    info = info[info['年份（年末）'] == year + 1]
    if len(info) == 0:
        return None
    else:
        return info['是否送转'].values[0]

def get_next_high_tran(code,year):
    """
    input:
        code:股票编号
        year:年份
    output:
        该股票在对应年份的次年是否高送转
        
    """
    info = tran_data[tran_data['股票编号'] == code]
    info = info[info['年份（年末）'] == year + 1]
    if len(info) == 0:
        return None
    else:
        return info['是否高送转'].values[0]

In [8]:
code_list = tran_data['股票编号'].values
year_list = tran_data['年份（年末）'].values

next_tran = []
next_high_tran = []
near2_tran = []
for code,year in zip(code_list,year_list):
    near2_tran.append(get_near2_tran(code,year))
    next_tran.append(get_next_tran(code,year))
    next_high_tran.append(get_next_high_tran(code,year))

# 时序因子，近两年送转比例，第一年年份没有该因子
tran_data['近两年送转比例平均'] = near2_tran

# 标签，第七年份是没有“次年是否送转/高送转”的标签
tran_data['次年是否送转'] = next_tran
tran_data['次年是否高送转'] = next_high_tran

In [10]:
tran_data.head(10)

,股票编号,年份（年末）,本年送转比例,是否送转,是否高送转,近两年送转比例平均,次年是否送转,次年是否高送转
0,1,1,0.00,0,0,NaN,0.0,0.0
1,1,2,0.00,0,0,0.000,1.0,0.0
2,1,3,0.25,1,0,0.125,1.0,0.0
3,1,4,0.22,1,0,0.235,1.0,0.0
4,1,5,0.08,1,0,0.150,1.0,0.0
5,1,6,0.18,1,0,0.130,1.0,0.0
6,1,7,0.30,1,0,0.240,NaN,NaN
7,2,1,0.00,0,0,NaN,0.0,0.0
8,2,2,0.00,0,0,0.000,0.0,0.0
9,2,3,0.00,0,0,0.000,0.0,0.0


从年数据和基本数据中，提取成长因子数据和基本因子数据

In [11]:
# 关键字，股票编号和年份
keys = data_year[['股票编号','年份（年末）']]

In [12]:
# 股本
cap_tot = data_year['实收资本(或股本)'].values

# 每股资本公积
capres = data_year['每股资本公积(元/股)'].values
# 每股未分配利润
prof = data_year['每股未分配利润(元/股)'].values

# 每股收益
eps = data_year['基本每股收益'].values
# 基本每股收益同比增长(%)
eps_growth = data_year['基本每股收益同必增长(%)'].values

# 每股净资产
per_oeu = data_year['每股净资产(元/股)'].values
# 每股净资产相对年初增长
per_oeu_growth = data_year['每股净资产相对年初增长(%)']

# 每股现金流
per_cash = data_year['每股现金流量净额(元/股)'].values

# 每股营业收入
per_rev = data_year['每股营业收入(元/股)'].values
# 营业收入同比增速
rev_growth = data_year['营业收入同必增长(%)'].values

# 净资产收益率
roe = data_year['净资产收益率(加权平均，%)'].values # 强调企业业的净资产创造利润的能力。 
# 净资产收益率（摊薄）同必增长(%)
roe_growth = data_year['净资产收益率（摊薄）同必增长(%)'].values

# 净利润同比增长
prof_growth = data_year['净利润同必增长(%)'].values


In [13]:
# 成长属性因子整合
grow_fac = keys.copy()

grow_fac['总股本'] = cap_tot

grow_fac['每股资本公积'] = capres
grow_fac['每股未分配利润'] = prof
grow_fac['基本每股收益'] = eps
grow_fac['基本每股收益同比增长(%)'] = eps_growth
grow_fac['每股净资产'] = per_oeu
grow_fac['每股净资产相对年初增长(%)'] = per_oeu_growth
grow_fac['每股现金流'] = per_cash
grow_fac['每股营业收入'] = per_rev
grow_fac['营业收入同比增速(%)'] = rev_growth
grow_fac['加权平均净资产收益率'] = roe
grow_fac['摊薄净资产收益率同比增长(%)'] = roe_growth
grow_fac['净利润同比增长'] = prof_growth

In [14]:
grow_fac.head(10)

,股票编号,年份（年末）,总股本,每股资本公积,每股未分配利润,基本每股收益,基本每股收益同比增长(%),每股净资产,每股净资产相对年初增长(%),每股现金流,每股营业收入,营业收入同比增速(%),加权平均净资产收益率,摊薄净资产收益率同比增长(%),净利润同比增长
0,1,1,595979092.0,0.1306,0.6765,0.4381,51.0690,1.9153,30.5768,-0.1278,2.3998,44.0040,25.67,14.2688,48.5989
1,1,2,595979092.0,0.2015,1.1579,0.6299,43.7800,2.5222,31.6869,0.5510,3.1231,30.1415,28.43,9.1834,44.1369
2,1,3,595979092.0,0.2015,1.6314,0.5048,-19.8603,3.0249,19.9310,0.3011,2.7169,-13.0055,18.19,-33.1844,-19.7378
3,1,4,595979092.0,0.2013,2.0567,0.7005,38.7678,3.4804,15.0583,-0.2822,2.1283,-21.6632,21.82,20.6153,38.7774
4,1,5,595979092.0,0.2013,2.0695,0.2631,-62.4411,3.5235,1.2384,0.2087,1.8078,-15.0603,7.59,-62.8976,-62.4382
5,1,6,595979092.0,0.2013,2.4189,0.5954,126.3018,4.0445,14.7864,3.2283,3.4552,91.1239,15.79,97.1331,126.2832
6,1,7,595979092.0,0.1996,3.2070,1.0453,75.5626,4.9023,21.2090,-0.6590,4.8738,41.0589,26.64,44.8338,75.5531
7,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# 读取基本数据
data_basic = pd.read_csv('Data/基础数据.csv',encoding = 'GBK',engine='python')
data_basic.head(10)

,股票编号,上市年限,所属行业,所属概念板块
0,1,26,房地产业,预增;小盘成长;标普道琼斯概念;地方国企;深圳市国资;粤港澳大湾区;粤港澳自贸区;预减;含B...
1,2,1,制造业,次新股;音乐产业;消费电子产业;合资企业;智能音箱
2,3,17,制造业,股权转让;融资融券;低价小市值;融资融券标的;成渝特区
3,4,22,批发和零售业,预增;地方国企
4,5,1,制造业,次新股
5,6,7,制造业,国家队;员工持股;超涨;重组
6,7,1,制造业,打板;超涨
7,8,3,制造业,可转债
8,9,17,制造业,充电桩;太赫兹;地方国企;天津市国资;预减;钨矿;超涨;超导;国家队;智能电网
9,10,1,制造业,高频PCB;科技龙头;基金重仓(季调);二线龙头;可转债;手机产业;电路板;珠三角;消费电子...


In [17]:
# 所属行业哑变量处理
industry = pd.get_dummies(data_basic['所属行业'])
data_basic_pro = pd.concat([data_basic, industry], axis = 1)
del industry

# 判断股票是否为次新股
data_basic_pro['是否次新股'] = data_basic['所属概念板块'].apply(lambda x:int('次新股' in str(x)))
# 将原非数值型信息删除
data_basic_pro.drop(axis = 1,columns = ['所属行业','所属概念板块'],inplace = True)

data_basic_pro.head(10)

,股票编号,上市年限,交通运输、仓储和邮政业,住宿和餐饮业,信息传输、软件和信息技术服务业,农、林、牧、渔业,制造业,卫生和社会工作,建筑业,房地产业,...,教育,文化、体育和娱乐业,水利、环境和公共设施管理业,电力、热力、燃气及水生产和供应业,科学研究和技术服务业,租赁和商务服务业,综合,采矿业,金融业,是否次新股
0,1,26,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,17,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,6,7,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,3,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,17,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# 成长因子和基本因子主键连接整合
factors = pd.merge(grow_fac, data_basic_pro, on = '股票编号')

# 基本数据中的上市年限是以第七年年末为基准，因此前面的年份的上市年限因子需要顺次减少
basic_year = factors['上市年限']
year_list = factors['年份（年末）']
real_year = basic_year - 7 + year_list

factors['上市年限'] = real_year

# 将上市年限为负数的记录删除（说明这个年份对应公司还没有上市），不应该有记录（即便有也基本是空值）
factors = factors[factors['上市年限'] > 0]
print('当前记录条数：%d'%len(factors))

当前记录条数：19076


In [20]:
factors.columns

Index(['股票编号', '年份（年末）', '总股本', '每股资本公积', '每股未分配利润', '基本每股收益', '基本每股收益同比增长(%)',
       '每股净资产', '每股净资产相对年初增长(%)', '每股现金流', '每股营业收入', '营业收入同比增速(%)',
       '加权平均净资产收益率', '摊薄净资产收益率同比增长(%)', '净利润同比增长', '上市年限', '交通运输、仓储和邮政业',
       '住宿和餐饮业', '信息传输、软件和信息技术服务业', '农、林、牧、渔业', '制造业', '卫生和社会工作', '建筑业',
       '房地产业', '批发和零售业', '教育', '文化、体育和娱乐业', '水利、环境和公共设施管理业',
       '电力、热力、燃气及水生产和供应业', '科学研究和技术服务业', '租赁和商务服务业', '综合', '采矿业', '金融业',
       '是否次新股'],
      dtype='object')

与行情数据整合

In [19]:
price_vol = pd.read_csv('temp/股价均价和波动率.csv',encoding = 'GBK',engine = 'python')

price_vol.head()

,股票编号,年份（年末）,股价（年末最后20个交易日收盘价加权平均）,股价年化波动率
0,1,1,6.186360,0.500901
1,1,2,5.812830,0.490998
2,1,3,7.159914,0.423839
3,1,4,9.088111,0.342857
4,1,5,13.947512,0.754430


In [22]:
fac_save = pd.merge(factors, price_vol, on = ['股票编号','年份（年末）'])

In [23]:
fac_save.head()

,股票编号,年份（年末）,总股本,每股资本公积,每股未分配利润,基本每股收益,基本每股收益同比增长(%),每股净资产,每股净资产相对年初增长(%),每股现金流,...,水利、环境和公共设施管理业,电力、热力、燃气及水生产和供应业,科学研究和技术服务业,租赁和商务服务业,综合,采矿业,金融业,是否次新股,股价（年末最后20个交易日收盘价加权平均）,股价年化波动率
0,1,1,595979092.0,0.1306,0.6765,0.4381,51.0690,1.9153,30.5768,-0.1278,...,0,0,0,0,0,0,0,0,6.186360,0.500901
1,1,2,595979092.0,0.2015,1.1579,0.6299,43.7800,2.5222,31.6869,0.5510,...,0,0,0,0,0,0,0,0,5.812830,0.490998
2,1,3,595979092.0,0.2015,1.6314,0.5048,-19.8603,3.0249,19.9310,0.3011,...,0,0,0,0,0,0,0,0,7.159914,0.423839
3,1,4,595979092.0,0.2013,2.0567,0.7005,38.7678,3.4804,15.0583,-0.2822,...,0,0,0,0,0,0,0,0,9.088111,0.342857
4,1,5,595979092.0,0.2013,2.0695,0.2631,-62.4411,3.5235,1.2384,0.2087,...,0,0,0,0,0,0,0,0,13.947512,0.754430


加入时序因子和标签

In [30]:
# 加入时序因子和标签
fac_save = pd.merge(fac_save, tran_data[['股票编号','年份（年末）',
                                       '本年送转比例',
                                       '近两年送转比例平均',
                                       '次年是否送转',
                                       '次年是否高送转']], on = ['股票编号','年份（年末）'])

In [33]:
print(fac_save.columns)
print('因子数据维度：',fac_save.shape)
print('无空值数据记录：%d'%len(fac_save.dropna()))

Index(['股票编号', '年份（年末）', '总股本', '每股资本公积', '每股未分配利润', '基本每股收益', '基本每股收益同比增长(%)',
       '每股净资产', '每股净资产相对年初增长(%)', '每股现金流', '每股营业收入', '营业收入同比增速(%)',
       '加权平均净资产收益率', '摊薄净资产收益率同比增长(%)', '净利润同比增长', '上市年限', '交通运输、仓储和邮政业',
       '住宿和餐饮业', '信息传输、软件和信息技术服务业', '农、林、牧、渔业', '制造业', '卫生和社会工作', '建筑业',
       '房地产业', '批发和零售业', '教育', '文化、体育和娱乐业', '水利、环境和公共设施管理业',
       '电力、热力、燃气及水生产和供应业', '科学研究和技术服务业', '租赁和商务服务业', '综合', '采矿业', '金融业',
       '是否次新股', '股价（年末最后20个交易日收盘价加权平均）', '股价年化波动率', '本年送转比例', '近两年送转比例平均',
       '次年是否送转', '次年是否高送转'],
      dtype='object')
因子数据维度： (19076, 41)
无空值数据记录：12811


In [34]:
fac_save.to_csv('temp/因子数据提取.csv',encoding = 'GBK',index = False)